In [10]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.node_parser import HTMLNodeParser
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.llms.gemini import Gemini

reader = SimpleDirectoryReader(input_dir="/home/dai/33/project/rag/data/clean_html",
                                  recursive=True)

documents = reader.load_data(show_progress=True)
node_parser = HTMLNodeParser(tags = ["p", "h1", "h2", "h3", "h4", "h5", "h6", "li", "b", "i", "u", "section", "text", "title"])
nodes = node_parser.get_nodes_from_documents(documents, show_progress=True)
nodes = [node for node in nodes if len(node.get_content()) > 0]
for node in nodes:
    #remove all the \n and \t
    node.text = node.text.replace("\n", " ").replace("\t", " ")

Loading files: 100%|██████████| 594/594 [00:00<00:00, 7141.12file/s]


Parsing nodes:   0%|          | 0/594 [00:00<?, ?it/s]

In [11]:
from llama_index.core.node_parser import HierarchicalNodeParser

node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[2048]
)

nodes = node_parser.get_nodes_from_documents(nodes, show_progress=True)
nodes = [node for node in nodes if len(node.get_content()) > 0]
len(nodes)

Parsing documents into nodes:   0%|          | 0/5031 [00:00<?, ?it/s]

5104

In [18]:
greater_size_docs = []
for i in range(len(nodes)):
    size = nodes[i].metadata['file_size']
    if size > 10000:
        greater_size_docs.append(nodes[i].metadata['file_name'])

len(greater_size_docs)

2110

In [12]:
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.core import Settings

Settings.embed_model = GeminiEmbedding(model_name='models/embedding-001')

index = VectorStoreIndex(nodes=nodes, embed_model=Settings.embed_model, show_progress=True)

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

InvalidArgument: 400 Request payload size exceeds the limit: 10000 bytes.